In [1]:
from LSTM import LSTM
import os
import pandas as pd
import math
import DataPreparationLSTM
import tensorflow as tf

2024-05-11 21:50:51.771882: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-11 21:50:52.420586: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-11 21:50:52.441183: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-11 21:50:55.614881: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
input_data = pd.read_csv('../data/SPY_data_5min.csv')
input_data['time'] = pd.to_datetime(input_data['time'])

# Filter trades within the time window from 09:30 to 16:00
input_data = input_data[(input_data['time'].dt.time >= pd.to_datetime('09:30').time()) & 
                 (input_data['time'].dt.time <= pd.to_datetime('16:00').time())]

input_data.drop(['Unnamed: 0','money','open','high','low'], axis=1, inplace=True)
input_data['time'] = pd.to_datetime(input_data['time']).dt.strftime('%H:%M')


In [ ]:
#Defining features
futures = [1,5]
models = []

models.append([1,20,False,0.01,20,20,0,0]) # future, lag, semi_variance, learning_rate, layer_1, layer_2, layer_3, layer_4
models.append([1, 20, True, 0.01, 40, 40, 0, 0])
models.append([1, 30, False, 0.01, 20, 20, 0, 0])
models.append([1, 30, True, 0.01, 20, 20, 0, 0])
models.append([5, 30, False, 0.01, 20, 40, 0, 0])
models.append([5, 30, True, 0.01, 20, 20, 0, 0])


In [ ]:
results = {}

for m in models:

    prepared_data = DataPreparationLSTM.DataPreparationLSTM(

                df = input_data,

                future = m[0],

                lag = m[1],

                min_max_scaler=True,

                log_transform=True,

                semi_variance=m[2],

                jump_detect=True,

                period_train= list(
                        [
                            pd.to_datetime("2006-01-01",format = "%Y-%m-%d"),
                            pd.to_datetime("2015-12-31",format = "%Y-%m-%d"),

                        ]
),

                period_validation = list(
                        [

                        pd.to_datetime("2013-01-01",format = "%Y-%m-%d"),
                        pd.to_datetime("2015-12-31",format = "%Y-%m-%d"),  

                                ]

                ),

                period_test= list(
                        [
                            pd.to_datetime("2016-01-01",format = "%Y-%m-%d"), 
                            pd.to_datetime("2018-12-31",format = "%Y-%m-%d"),

                        ]
                    ),
)


    prepared_data.prepare_all()
    
    tf.keras.backend.clear_session()

    
    model = LSTM(
                train_matrix = prepared_data.x_train,
                train_y = prepared_data.y_train,
                test_matrix= prepared_data.x_test,
                test_y = prepared_data.y_test,
                epochs = 50,
                learning_rate = m[3],
                layer_one= m[4],
                layer_two= m[5],
                layer_three=m[6],
                layer_four= m[7]
            )


    model.train_lstm()
    model.predict_lstm()
            
    prediction_train = model.prediction_train
    prediction_test = model.prediction_test        
    target_scaler = prepared_data.applied_scaler_targets
        
    prediction_train_normal = prepared_data.back_transformation(prediction_train)
    prediction_test_normal = prepared_data.back_transformation(prediction_test)
        
    train_y_normal = prepared_data.back_transformation(model.train_y)
    test_y_normal = prepared_data.back_transformation(model.test_y)
        
    model.prediction_train = prediction_train_normal
    model.prediction_test = prediction_test_normal
    model.test_y = test_y_normal
    model.train_y = train_y_normal
    
    model.make_accuracy_measures()

    results[f'{m[0]}_{m[1]}_{m[2]}'] = [model.train_accuracy, model.test_accuracy]

In [ ]:

additional_models = []
additional_models.append([5,20,True, 0.01, 20, 40, 0, 0])
additional_models.append([5,20,False, 0.01, 20, 40, 0, 0])
additional_models.append([20,20, True, 0.01, 40, 20, 0, 0])
additional_models.append([20, 20, False, 0.01, 40, 20, 0, 0])

for m in additional_models:

    prepared_data = DataPreparationLSTM.DataPreparationLSTM(

                df = input_data,

                future = m[0],

                lag = m[1],

                min_max_scaler=True,

                log_transform=True,

                semi_variance=m[2],

                jump_detect=True,

                period_train= list(
                        [
                            pd.to_datetime("2006-01-01",format = "%Y-%m-%d"),
                            pd.to_datetime("2015-12-31",format = "%Y-%m-%d"),

                        ]
),

                period_validation = list(
                        [

                        pd.to_datetime("2013-01-01",format = "%Y-%m-%d"),
                        pd.to_datetime("2015-12-31",format = "%Y-%m-%d"),  

                                ]

                ),

                period_test= list(
                        [
                            pd.to_datetime("2016-01-01",format = "%Y-%m-%d"), 
                            pd.to_datetime("2018-12-31",format = "%Y-%m-%d"),

                        ]
                    ),
)


    prepared_data.prepare_all()
    
    tf.keras.backend.clear_session()

    
    model = LSTM(
                train_matrix = prepared_data.x_train,
                train_y = prepared_data.y_train,
                test_matrix= prepared_data.x_test,
                test_y = prepared_data.y_test,
                epochs = 50,
                learning_rate = m[3],
                layer_one= m[4],
                layer_two= m[5],
                layer_three=m[6],
                layer_four= m[7]
            )


    model.train_lstm()
    model.predict_lstm()
            
    prediction_train = model.prediction_train
    prediction_test = model.prediction_test        
    target_scaler = prepared_data.applied_scaler_targets
        
    prediction_train_normal = prepared_data.back_transformation(prediction_train)
    prediction_test_normal = prepared_data.back_transformation(prediction_test)
        
    train_y_normal = prepared_data.back_transformation(model.train_y)
    test_y_normal = prepared_data.back_transformation(model.test_y)
        
    model.prediction_train = prediction_train_normal
    model.prediction_test = prediction_test_normal
    model.test_y = test_y_normal
    model.train_y = train_y_normal
    
    model.make_accuracy_measures()

    results[f'{m[0]}_{m[1]}_{m[2]}'] = [model.train_accuracy, model.test_accuracy]

In [11]:
results = {}
additional_models = []
additional_models.append([20,30, True, 0.01, 20, 40, 0, 0])
additional_models.append([20, 30, False, 0.01, 20, 40, 0, 0])
for m in additional_models:

    prepared_data = DataPreparationLSTM.DataPreparationLSTM(

                df = input_data,

                future = m[0],

                lag = m[1],

                min_max_scaler=True,

                log_transform=True,

                semi_variance=m[2],

                jump_detect=True,

                period_train= list(
                        [
                            pd.to_datetime("2006-01-01",format = "%Y-%m-%d"),
                            pd.to_datetime("2015-12-31",format = "%Y-%m-%d"),

                        ]
),

                period_validation = list(
                        [

                        pd.to_datetime("2013-01-01",format = "%Y-%m-%d"),
                        pd.to_datetime("2015-12-31",format = "%Y-%m-%d"),  

                                ]

                ),

                period_test= list(
                        [
                            pd.to_datetime("2016-01-01",format = "%Y-%m-%d"), 
                            pd.to_datetime("2018-12-31",format = "%Y-%m-%d"),

                        ]
                    ),
)


    prepared_data.prepare_all()
    
    tf.keras.backend.clear_session()

    
    model = LSTM(
                train_matrix = prepared_data.x_train,
                train_y = prepared_data.y_train,
                test_matrix= prepared_data.x_test,
                test_y = prepared_data.y_test,
                epochs = 50,
                learning_rate = m[3],
                layer_one= m[4],
                layer_two= m[5],
                layer_three=m[6],
                layer_four= m[7]
            )


    model.train_lstm()
    model.predict_lstm()
            
    prediction_train = model.prediction_train
    prediction_test = model.prediction_test        
    target_scaler = prepared_data.applied_scaler_targets
        
    prediction_train_normal = prepared_data.back_transformation(prediction_train)
    prediction_test_normal = prepared_data.back_transformation(prediction_test)
        
    train_y_normal = prepared_data.back_transformation(model.train_y)
    test_y_normal = prepared_data.back_transformation(model.test_y)
        
    model.prediction_train = prediction_train_normal
    model.prediction_test = prediction_test_normal
    model.test_y = test_y_normal
    model.train_y = train_y_normal
    
    model.make_accuracy_measures()

    results[f'{m[0]}_{m[1]}_{m[2]}'] = [model.train_accuracy, model.test_accuracy]

/home/korisnik/Desktop/Projects/VolatilityPredictionMatf/models/DataPreparationLSTM.py:118: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_output["Target"] = help_df.mean(axis=1)


Epoch 1/50
77/77 [==============================] - 5s 26ms/step - loss: 0.0130 - val_loss: 0.0139
Epoch 2/50
77/77 [==============================] - 2s 22ms/step - loss: 0.0099 - val_loss: 0.0137
Epoch 3/50
77/77 [==============================] - 2s 21ms/step - loss: 0.0079 - val_loss: 0.0100
Epoch 4/50
77/77 [==============================] - 2s 21ms/step - loss: 0.0069 - val_loss: 0.0087
Epoch 5/50
77/77 [==============================] - 2s 20ms/step - loss: 0.0058 - val_loss: 0.0090
Epoch 6/50
77/77 [==============================] - 2s 21ms/step - loss: 0.0057 - val_loss: 0.0101
Epoch 7/50
77/77 [==============================] - 2s 21ms/step - loss: 0.0056 - val_loss: 0.0105
Epoch 8/50
77/77 [==============================] - 2s 21ms/step - loss: 0.0054 - val_loss: 0.0095
Epoch 9/50
77/77 [==============================] - 2s 21ms/step - loss: 0.0056 - val_loss: 0.0102
Epoch 10/50
77/77 [==============================] - 2s 21ms/step - loss: 0.0055 - val_loss: 0.0133
Epoch 11/

/home/korisnik/Desktop/Projects/VolatilityPredictionMatf/models/DataPreparationLSTM.py:118: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_output["Target"] = help_df.mean(axis=1)


Epoch 1/50
77/77 [==============================] - 4s 26ms/step - loss: 0.0138 - val_loss: 0.0109
Epoch 2/50
77/77 [==============================] - 2s 20ms/step - loss: 0.0088 - val_loss: 0.0109
Epoch 3/50
77/77 [==============================] - 2s 21ms/step - loss: 0.0082 - val_loss: 0.0150
Epoch 4/50
77/77 [==============================] - 2s 20ms/step - loss: 0.0075 - val_loss: 0.0128
Epoch 5/50
77/77 [==============================] - 2s 20ms/step - loss: 0.0067 - val_loss: 0.0140
Epoch 6/50
77/77 [==============================] - 2s 20ms/step - loss: 0.0070 - val_loss: 0.0149
Epoch 7/50
77/77 [==============================] - 2s 21ms/step - loss: 0.0072 - val_loss: 0.0132
Epoch 8/50
77/77 [==============================] - 2s 20ms/step - loss: 0.0066 - val_loss: 0.0125
Epoch 9/50
77/77 [==============================] - 2s 21ms/step - loss: 0.0064 - val_loss: 0.0119
Epoch 10/50
77/77 [==============================] - 2s 21ms/step - loss: 0.0062 - val_loss: 0.0146
Epoch 11/

In [12]:
for key,value in results.items():
    print(key)
    print(value)
    

20_30_True
[{'MSE': 9.103866965530443e-09, 'MAE': 3.2824021509816466e-05, 'RSquared': 0.6033454104872604}, {'MSE': 8.573308306258514e-10, 'MAE': 1.884749923692691e-05, 'RSquared': 0.3954377644622441}]
20_30_False
[{'MSE': 1.3837995377368282e-08, 'MAE': 3.792367633566892e-05, 'RSquared': 0.39707990056625464}, {'MSE': 8.883855747257419e-10, 'MAE': 1.9844380466020215e-05, 'RSquared': 0.3735389538206423}]


In [13]:
output_directory = "../output/LSTM_outputdata"
os.makedirs(output_directory,exist_ok=True)
for name, value in results.items():
    output_file_path = os.path.join(output_directory,f'{name}.txt')
          
    with open(output_file_path, 'w') as output_file:
        output_file.write("Train Accuracy  \n")
        output_file.write(f"{value[0]} \n")
        output_file.write("Test Accuracy  \n")
        output_file.write(f"{value[1]} \n ")
